<a href="https://colab.research.google.com/github/engineerjkk/FaceNet/blob/main/FaceNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import torch
import torch.nn as nn
from torch.nn.modules.distance import PairwiseDistance

class TripletLoss(nn.Module):
  def __init__(self, margin):
    super(TripletLoss,self).__init__()
    self.margin=margin
    self.pdist=PairwiseDistance(p=2)

  def forward(self,anchor,positive,negative):
    pos_dist=self.pdist.forward(anchor,positive)
    neg_dist=self.pdist.forward(anchor,negative)

    hinge_dist=torch.clamp(self.margin+pos_dist-neg_dist,min=0.0)
    loss=torch.mean(hinge_dist)

    return loss

In [10]:
from torchvision.models import resnet18
from torch.nn import functional as F
from torchvision.models import resnet18

class FaceNet_ResNet18(nn.Module):
  def __init__(self, embedding_dimension=128,pretrained=False):  #embedding dimension은 128로 설정을 해준다. 당시엔 128이 가장 잘됐었다. pretrained를 사용하면 반칙이죠.
    super().__init__()
    self.model=resnet18(pretrained=pretrained)

    #embedding
    input_features_fc_layer=self.model.fc.in_features #fc layer 채널수얻기, 128 dimension을 return할 것이고.
    self.model.fc=nn.Linear(input_features_fc_layer,embedding_dimension, bias=False)

  def forward(self, images):
    embedding=self.model(images)
    embedding=F.normalize(embedding,p=2,dim=1)#normalize
    return embedding

In [11]:
model=FaceNet_ResNet18()
criterion=TripletLoss(margin=0.2)

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [12]:
B,C,H,W=4,3,224,224

anchor=torch.randn(B,C,H,W)
positive=torch.randn(B,C,H,W)
negative=torch.randn(B,C,H,W)

## 실제 학습을 시키고 싶다면 anchor, positive, negative 에다가 실제 이미지를 넣어주면 된다.

In [13]:
a_embed=model(anchor)
p_embed=model(positive)
n_embed=model(negative)

In [15]:
print(a_embed.shape, p_embed.shape, n_embed.shape)

torch.Size([4, 128]) torch.Size([4, 128]) torch.Size([4, 128])


In [17]:
loss=criterion(a_embed, p_embed, n_embed)

In [18]:
loss

tensor(0.2011, grad_fn=<MeanBackward0>)

In [ ]:
for data in dataloader:
  a=model(data['anchor'])
  p=model(data['positive'])
  n=model(data['negative'])

In [19]:
criterion2=nn.TripletMarginLoss(margin=0.2)

In [20]:
loss2=criterion2(a_embed,p_embed,n_embed)

In [21]:
loss2

tensor(0.2011, grad_fn=<MeanBackward0>)